## Tree Edit Distance

The tree edit distance ([Zhang and Shasha, 1989](https://doi.org/10.1137/0218082)) describes the distance between trees as the number of node deletions, node insertions, and node relabelings we need to apply to transform one tree into another. According to their definition, node deletions push all children of the deleted node up to its parent and insertions push child nodes down to be children of the newly inserted node. The following examples illustrate the edits.

In [1]:
import edist.tree_utils as tree_utils
import edist.tree_edits as tree_edits

nodes = ['a', 'b', 'c']
adj   = [[1, 2], [], []]

edit  = tree_edits.Replacement(1, 'd')
nodes_post, adj_post = edit.apply(nodes, adj)
print('If we relabel node 1 in tree %s with a d, we obtain the tree %s.' % (tree_utils.tree_to_string(nodes, adj), tree_utils.tree_to_string(nodes_post, adj_post)))

edit  = tree_edits.Insertion(0, 0, 'd', 1)
nodes_post, adj_post = edit.apply(nodes, adj)
print('If we insert a new node with label d as first child of node 0 tree %s and use one child as new grandchild, we obtain the tree %s.' % (tree_utils.tree_to_string(nodes, adj), tree_utils.tree_to_string(nodes_post, adj_post)))

nodes = nodes_post
adj   = adj_post
edit  = tree_edits.Deletion(1)
nodes_post, adj_post = edit.apply(nodes, adj)
print('If delete node 1 in tree %s, we obtain the tree %s.' % (tree_utils.tree_to_string(nodes, adj), tree_utils.tree_to_string(nodes_post, adj_post)))


If we relabel node 1 in tree a(b, c) with a d, we obtain the tree a(d, c).
If we insert a new node with label d as first child of node 0 tree a(b, c) and use one child as new grandchild, we obtain the tree a(d(b), c).
If delete node 1 in tree a(d(b), c), we obtain the tree a(b, c).


Note that we denote trees in a node list/adjacency list format. For example, the tree $a(b, c)$ has the node list `[a, b, c]` (in depth-first-search order) and node 0 has nodes 1 and 2 as children, whereas nodes 1 and 2 have no children of their own, which results in the adjacency list `[[1, 2], [], []]`.

Now, given these edit definitions, we can infer that the edit distance between the trees $a(b(c, d), e)$ and $a(c(d))$ is $3$, because we can relabel $b$ into $c$, leave $a$ and $d$ as they were, and delete $c$ and $e$ to transform the former tree into the latter. This example also illustrates that the tree edit distance and the sequence edit distance are _not_ the same thing, because the sequence edit distance on the node list would only be $2$.

In [2]:
import edist.ted as ted
import edist.sed as sed
x_nodes = ['a', 'b', 'c', 'd', 'e']
x_adj   = [[1, 4], [2, 3], [], [], []]
y_nodes = ['a', 'c', 'd']
y_adj   = [[1], [2], []]
print('The tree edit distance between tree %s and tree %s is %d.' % (tree_utils.tree_to_string(x_nodes, x_adj), tree_utils.tree_to_string(y_nodes, y_adj), ted.standard_ted(x_nodes, x_adj, y_nodes, y_adj)))
print('By contrast, the sequence edit distance on the node lists would be %d.' % (sed.standard_sed(x_nodes, y_nodes)))

The tree edit distance between tree a(b(c, d), e) and tree a(c(d)) is 3.
By contrast, the sequence edit distance on the node lists would be 2.


We can also use backtracing to infer exactly which node has been aligned with which other node.

In [3]:
print('The following nodes have been aligned.')
alignment = ted.standard_ted_backtrace(x_nodes, x_adj, y_nodes, y_adj)
print(alignment.render(x_nodes, y_nodes))

The following nodes have been aligned.
a [0] vs. a [0]
b [1] vs. c [1]
c [2] vs. -
d [3] vs. d [2]
e [4] vs. -


We can also infer the tree edits necessary to transform x into y via the tree edits module.

In [4]:
script = tree_edits.alignment_to_script(alignment, x_nodes, x_adj, y_nodes, y_adj)
x_nodes2, x_adj2 = script.apply(x_nodes, x_adj)
print('The following edits transform %s to %s: %s' % (tree_utils.tree_to_string(x_nodes, x_adj), tree_utils.tree_to_string(x_nodes2, x_adj2), str(script)))

The following edits transform a(b(c, d), e) to a(c(d)): [rep(1, c), del(4), del(2)]


It is also possible to use custom distance functions between tree nodes. For example, assume we wish to compare algebraic expressions but do only wish to compare matching types of nodes.

In [5]:
x_nodes = ['+', 3, '*', 5, 2]
x_adj   = [[1, 2], [], [3, 4], [], []]
y_nodes = ['*', 2, '*', 2, 3]
y_adj   = [[1, 2], [], [3, 4], [], []]

# define the distance between nodes in algebraic expressions
import numpy as np
def delta(x, y):
    if(x is None or y is None):
        return 1.
    if(x in ['+', '*'] or y in ['+', '*']):
        if(x == y):
            return 0.
        else:
            # we forbid alignments of algebraic operators with
            # other types by assigning an infinite cost
            return np.inf
    # at this point, we now that both entries are numbers
    return abs(x - y) / max(x, y)

print('The tree edit distance between the expressions %s and %s is %g' % (tree_utils.tree_to_string(x_nodes, x_adj), tree_utils.tree_to_string(y_nodes, y_adj), ted.ted(x_nodes, x_adj, y_nodes, y_adj, delta)))
alignment = ted.ted_backtrace(x_nodes, x_adj, y_nodes, y_adj, delta)
print('The following nodes have been aligned:')
print(alignment.render(x_nodes, y_nodes, delta))

The tree edit distance between the expressions +(3, *(5, 2)) and *(2, *(2, 3)) is 3.26667
The following nodes have been aligned:
+ [0] vs. -: 1.0
- vs. * [0]: 1.0
3 [1] vs. 2 [1]: 0.3333333333333333
* [2] vs. * [2]: 0.0
5 [3] vs. 2 [3]: 0.6
2 [4] vs. 3 [4]: 0.3333333333333333
